In [ ]:
import json
import os
import random
import keras
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, Lambda, ELU, Permute
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from PIL import Image, ImageDraw, ImageOps
import matplotlib.pyplot as plt

random.seed(1337)

In [ ]:
CLASSES = {
   'R': [1, 0, 0, 0],
   'Y': [0, 1, 0, 0],
   'G': [0, 0, 1, 0],
   'U': [0, 0, 0, 1],
}
BASE_PATH = 'lights'
TRAIN = 0.8

In [ ]:
truth = json.loads(open('truth.json').read())
FACTOR = 4
filtered_truth = {}
for name, votes in truth.items():
    R = votes['R']
    Y = votes['Y']
    G = votes['G']
    U = votes['U'] + votes['X']
    cls = None
    
    if U > FACTOR * R and U > FACTOR * Y and U > FACTOR * G:
        cls = 'U'
    else:
        if R > FACTOR * (Y + G) and R > FACTOR * U:
            cls = 'R'
        elif Y > FACTOR * (R + G) and Y > FACTOR * U:
            cls = 'Y'
        elif G > FACTOR * (R + Y) and G > FACTOR * U:
            cls = 'G'
    if cls:
        filtered_truth[name] = CLASSES[cls]



In [ ]:
names = filtered_truth.keys()
random.shuffle(names)

train_names = names[:int(len(names) * TRAIN)]
test_names = names[int(len(names) * TRAIN):]

x_train = []
y_train = []
x_test = []
y_test = []

sxs = []
def augment(img):
    safe_x = 30
    safe_y = 60
    w, h = img.size
    
    sx = w / 2 - 32 + random.uniform(-safe_x/2, safe_x/2)
    sy = h / 2 - 64 + random.uniform(-safe_y/2, safe_y/2)
    sxs.append(sx)
    
    r = img.crop((sx, sy, sx+64, sy+128))
    if random.random() > 0.5:
        r = ImageOps.mirror(r)
        
        
    imga = np.asarray(r, dtype=np.uint8)
    imga = imga.astype(np.float32) / 256.
    
    return imga

for name in train_names:
    img = Image.open(os.path.join(BASE_PATH, name))
    cls = filtered_truth[name]
    
    for k in xrange(32):
        imga = augment(img.copy())
        x_train.append(imga)
        y_train.append(cls)

for name in test_names:
    img = Image.open(os.path.join(BASE_PATH, name))
    cls = filtered_truth[name]
    
    for k in xrange(32):
        imga = augment(img.copy())
        x_test.append(imga)
        y_test.append(cls)

x_train = np.array(x_train)
y_train = np.array(y_train)
    
x_test = np.array(x_test)
y_test = np.array(y_test)

print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

In [ ]:
def plot_grid(images, size=8):
    plt.figure(figsize=(20,20))
    for i in range(size**2):
        plt.subplot(size, size, i + 1)
        plt.imshow(random.choice(images), cmap='binary')
        plt.axis('off')
    plt.show()
    
plot_grid(x_train)

In [ ]:
H, W = 128, 64

input_ = x = Input(shape=(H, W, 3), name="input_img")
x = Permute((2, 1, 3))(x) # (W, H, D)
x = Conv2D(8, (3,3), padding='same', activation='relu')(x)
x = Conv2D(8, (3,3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
for k in [16, 16, 16]:
    x = Conv2D(k, (3, 3), padding="same", activation='relu')(x)
    x = Conv2D(k, (3, 3), padding="same", activation='relu')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)
    #x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)

m = Model(inputs=[input_], outputs=[x])
opt = keras.optimizers.Adam(lr=0.0001)
m.compile(loss=keras.losses.categorical_crossentropy,
          metrics=['accuracy'],
          optimizer=opt)

m.summary()

m.fit(x_train, y_train, batch_size=16, epochs=16, verbose=1,
          validation_data=(x_test, y_test), shuffle=True)
score = m.evaluate(x_test, y_test, verbose=0)
print score
m.save("lights.keras")